# 1 Gráfico de Barras: Producción de Energía Renovable por Fuente

In [2]:
import os
import pandas as pd

ruta_base = os.getcwd()
ruta_datos = os.path.join(ruta_base, "..", "datos")


archivos = [
    ("Hidráulica", "05 hydropower-consumption.csv"),
    ("Eólica", "08 wind-generation.csv"),
    ("Solar", "12 solar-energy-consumption.csv"),
    ("Geotérmica", "17 installed-geothermal-capacity.csv"),
    ("Biocombustibles", "16 biofuel-production.csv")
]


continentes = ["World", "Africa", "Asia", "Europe", 
               "North America", "South America", "Oceania"]


paises = {
    "World": [],
    "Africa": ["Algeria", "Angola", "Benin", "Botswana", "Burkina Faso", "Burundi", "Cameroon", "Cape Verde", "Central African Republic", "Chad", "Comoros", "Congo", "Democratic Republic of Congo", "Djibouti", "Egypt", "Equatorial Guinea", "Eritrea", "Eswatini", "Ethiopia", "Gabon", "Gambia", "Ghana", "Guinea", "Guinea-Bissau", "Ivory Coast", "Kenya", "Lesotho", "Liberia", "Libya", "Madagascar", "Malawi", "Mali", "Mauritania", "Mauritius", "Morocco", "Mozambique", "Namibia", "Niger", "Nigeria", "Rwanda", "Sao Tome and Principe", "Senegal", "Seychelles", "Sierra Leone", "Somalia", "South Africa", "Sudan", "Tanzania", "Togo", "Tunisia", "Uganda", "Zambia", "Zimbabwe"],
    "Asia": ["Afghanistan", "Armenia", "Azerbaijan", "Bahrain", "Bangladesh", "Bhutan", "Brunei", "Cambodia", "China", "Cyprus", "Georgia", "India", "Indonesia", "Iran", "Iraq", "Israel", "Japan", "Jordan", "Kazakhstan", "Kuwait", "Kyrgyzstan", "Laos", "Lebanon", "Malaysia", "Maldives", "Mongolia", "Myanmar", "Nepal", "North Korea", "Oman", "Pakistan", "Palestine", "Philippines", "Qatar", "Saudi Arabia", "Singapore", "South Korea", "Sri Lanka", "Syria", "Taiwan", "Tajikistan", "Thailand", "Timor", "Turkey", "Turkmenistan", "United Arab Emirates", "Uzbekistan", "Vietnam", "Yemen"],  
    "Europe": ["Austria", "Belarus", "Belgium", "Bulgaria", "Croatia", "Czechia", "Denmark", "Estonia", "Finland", "France", "Germany", "Greece", "Hungary", "Iceland", "Ireland", "Italy", "Kosovo", "Latvia", "Lithuania", "Luxembourg", "Malta", "Moldova", "Montenegro", "Netherlands", "North Macedonia", "Norway", "Poland", "Portugal", "Romania", "Russia", "Serbia", "Slovakia", "Slovenia", "Spain", "Sweden", "Switzerland", "Ukraine", "United Kingdom"],
    "North America": ["Canada", "Mexico", "United States", "United States Virgin Islands"],
    "South America": ["Argentina", "Brazil", "Chile", "Colombia", "Ecuador", "Peru", "Uruguay", "Venezuela"],
    "Oceania": ["Australia", "New Zealand", "Fiji", "Papua New Guinea", "Samoa", "Solomon Islands", "Tonga", "Vanuatu"]
}

entidades_validas = continentes + sum(paises.values(), [])

df_unificado = pd.DataFrame()

for nombre, archivo in archivos:
    ruta = os.path.join(ruta_datos, archivo)
    
    if not os.path.exists(ruta):
        print(f"No se encontró el archivo: {ruta}")
        continue

    df = pd.read_csv(ruta)

    if "Year" not in df.columns or "Entity" not in df.columns:
        continue

    col_valor = "Value" if "Value" in df.columns else df.columns[-1]

    df = df[df["Entity"].isin(entidades_validas)]

    def obtener_continente(entity):
        for cont, paises_lista in paises.items():
            if entity in paises_lista or entity == cont:
                return cont
        return None


    df["Continent"] = df["Entity"].apply(obtener_continente)


    if nombre == "Geotérmica":
        df[col_valor] = df[col_valor] / 1_000_000

    df_filtrado = df[["Year", "Entity","Continent", col_valor]].rename(columns={col_valor: nombre})

    if df_unificado.empty:
        df_unificado = df_filtrado
    else:
        df_unificado = pd.merge(df_unificado, df_filtrado, on=["Year", "Entity", "Continent"], how="outer")

ruta_salida = os.path.join(ruta_datos, "datos_grafica1.csv")
df_unificado.to_csv(ruta_salida, index=False)
print(f"Archivo generado en: {ruta_salida}")

Archivo generado en: c:\Users\aleja\Desktop\EnergyMax\transicion-energetica\dashboard\..\datos\datos_grafica1.csv


# 2 Gráfico de Torta: Participación de Energías Renovables

In [ ]:
import os
import pandas as pd

ruta_base = os.getcwd()
ruta_datos = os.path.join(ruta_base, "..", "datos")



archivos = [
    ("Total_Renovables", "04 share-electricity-renewables.csv"),
    ("Eólica", "11 share-electricity-wind.csv"),
    ("Solar", "15 share-electricity-solar.csv"),
    ("Hidráulica", "07 share-electricity-hydro.csv"),
]

continentes = ["World", "Africa", "Asia", "Europe", 
               "North America", "South America", "Oceania"]


df_unificado = pd.DataFrame()

for nombre, archivo in archivos:
    ruta = os.path.join(ruta_datos, archivo)
    
    if not os.path.exists(ruta):
        print(f"No se encontró el archivo: {ruta}")
        continue

    df = pd.read_csv(ruta)

    if "Year" not in df.columns or "Entity" not in df.columns:
        continue

    col_valor = "Value" if "Value" in df.columns else df.columns[-1]

    df = df[df["Entity"].isin(continentes)]



    df[col_valor] = df[col_valor].astype(str)
    df[col_valor] = df[col_valor].str.replace(',', '.', regex=False)        
    df[col_valor] = df[col_valor].astype(float)   

   


    df_filtrado = df[["Year", "Entity", col_valor]].rename(columns={col_valor: nombre})


    if df_unificado.empty:
        df_unificado = df_filtrado
    else:
        df_unificado = pd.merge(df_unificado, df_filtrado, on=["Year", "Entity"], how="outer")

ruta_salida = os.path.join(ruta_datos, "datos_grafica2.csv")
df_unificado.to_csv(ruta_salida, index=False)
print(f"Archivo generado: {ruta_salida}")



Archivo generado: c:\Users\aleja\Desktop\EnergyMax\transicion-energetica\dashboard\..\datos\datos_grafica2.csv


# 3 Gráfico de Líneas: Tendencia en la Capacidad Instalada

In [1]:
import os
import pandas as pd

ruta_base = os.getcwd()
ruta_datos = os.path.join(ruta_base, "..", "datos")

archivos = [
    ("Eólica", "09 cumulative-installed-wind-energy-capacity-gigawatts.csv"),
    ("Solar", "13 installed-solar-PV-capacity.csv"),
    ("Geotérmica", "17 installed-geothermal-capacity.csv")
]

continentes = ["World", "Africa", "Asia", "Europe", 
               "North America", "South America", "Oceania"]


paises = {
    "World": [],
    "Africa": [ "Algeria", "Egypt", "Morocco", "South Africa", "Tunisia", "Ethiopia", "Kenya"],
    "Asia": ["China", "India", "Iran", "Japan", "Jordan", "Pakistan", "Philippines","South Korea", "Taiwan", "Thailand", "Israel", "Malaysia", "Vietnam", "Indonesia"],
    "Europe": ["Austria", "Belgium", "Bulgaria", "Croatia", "Czechia", "Denmark", "Finland","France", "Germany", "Greece", "Hungary", "Ireland", "Italy", "Netherlands","Norway", "Poland", "Portugal", "Romania", "Russia", "Slovakia", "Spain","Sweden", "Switzerland", "Turkey", "Ukraine", "United Kingdom", "Iceland"],
    "North America": ["Canada", "United States", "Mexico", "Costa Rica", "Honduras", "El Salvador", "Guatemala", "Nicaragua"],
    "South America": ["Argentina", "Brazil", "Chile", "Uruguay", "Other South & Central America","South & Central America"],
    "Oceania": ["Australia", "New Zealand", "Papua New Guinea"],
}

entidades_validas = continentes + sum(paises.values(), [])


df_unificado = pd.DataFrame()

for nombre, archivo in archivos:
    ruta = os.path.join(ruta_datos, archivo)

    if not os.path.exists(ruta):
        print(f"No se encontró el archivo: {ruta}")
        continue

    df = pd.read_csv(ruta)
    if "Year" not in df.columns or "Entity" not in df.columns:
        continue

    col_valor = "Value" if "Value" in df.columns else df.columns[-1]

    df = df[df["Entity"].isin(entidades_validas)]

    def obtener_continente(entity):
        for cont, paises_lista in paises.items():
            if entity in paises_lista or entity == cont:
                return cont
        return None

    df["Continent"] = df["Entity"].apply(obtener_continente)

    df_filtrado = df[["Year", "Entity","Continent", col_valor]].rename(columns={col_valor: nombre})

    if df_unificado.empty:
        df_unificado = df_filtrado
    else:
        df_unificado = pd.merge(df_unificado, df_filtrado, on=["Year", "Entity", "Continent"], how="outer")



ruta_salida = os.path.join(ruta_datos, "datos_grafica3.csv")
df_unificado.to_csv(ruta_salida, index=False)
print(f"Archivo generado: {ruta_salida}")


Archivo generado: c:\Users\aleja\Desktop\EnergyMax\transicion-energetica\dashboard\..\datos\datos_grafica3.csv


# 4 Gráfico de Área: Comparación entre Consumo de Energía Renovable y Convencional 

In [2]:
import os
import pandas as pd
import numpy as np

ruta_base = os.getcwd()
ruta_datos = os.path.join(ruta_base, "..", "datos")


df_conv = pd.read_excel(os.path.join(ruta_datos, "18 conventionales energy.xlsx"))


df_long = df_conv.melt(
    id_vars=["Entity", "Product", "Flow"],
    var_name="Year",
    value_name="Value"
)


df_filtered = df_long[df_long['Product'].isin(['Fossil fuels', 'Nuclear'])]


df_pivot = df_filtered.pivot_table(
    index=["Entity", "Year"],
    columns="Product",
    values="Value",
    aggfunc="first"
).reset_index()


col_fossil  = [c for c in df_pivot.columns if "Fossil fuels" in c][0]
col_nuclear = [c for c in df_pivot.columns if "Nuclear" in c][0]


for col in [col_fossil, col_nuclear]:
    df_pivot[col] = (
        df_pivot[col]
          .astype(str)
          .replace("..", np.nan)
          .str.replace(",", ".")
          .str.replace(" ", "")
          .astype(float)
    )


df_pivot["Combustibles fósiles (TWh)"] = df_pivot[col_fossil] / 1000
df_pivot["Nuclear (TWh)"] = df_pivot[col_nuclear]          / 1000

df_conv_final = df_pivot[['Entity','Year','Combustibles fósiles (TWh)','Nuclear (TWh)']]


df_ren = pd.read_csv(os.path.join(ruta_datos, "02 modern-renewable-energy-consumption.csv"))


cols_ren = {
    'Geo Biomass Other - TWh': 'Geotérmica/Biomasa (TWh)',
    'Solar Generation - TWh':  'Solar (TWh)',
    'Wind Generation - TWh':   'Eólica (TWh)',
    'Hydro Generation - TWh':  'Hidroeléctrica (TWh)',
}

df_ren = df_ren[['Entity', 'Year'] + list(cols_ren.keys())]
df_ren.rename(columns=cols_ren, inplace=True)


df_total = pd.merge(df_conv_final, df_ren, on=['Entity','Year'], how='outer').fillna(0)


ruta_salida = os.path.join(ruta_datos, "datos_grafica4.csv")
df_total.to_csv(ruta_salida, index=False)
print("Archivo generado:", ruta_salida)



Archivo generado: c:\Users\aleja\Desktop\EnergyMax\transicion-energetica\dashboard\..\datos\datos_grafica4.csv
